In [1]:
import pandas as pd

import os
os.chdir(r"C:\jupyter_notebook\input")

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
test_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,부산광역시,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


<span style=color:red>
    <h2>
    인덱스 확인
    </h2>
</span>

In [48]:
test_df['transaction_id']

0       1145756
1       1198704
2       1222384
3       1179897
4       1223091
         ...   
5458    1174640
5459    1175575
5460    1157024
5461    1136863
5462    1218921
Name: transaction_id, Length: 5463, dtype: int64

In [46]:
submission['transaction_id']

0       1145756
1       1198704
2       1222384
3       1179897
4       1223091
         ...   
5458    1174640
5459    1175575
5460    1157024
5461    1136863
5462    1218921
Name: transaction_id, Length: 5463, dtype: int64

<span style=color:red>
    <h2>
    수정을 용이하게 하기 위한 컬럼명 변경
    </h2>
</span>

In [4]:
train_df.rename(columns = {'city' : '도시',"dong" : "동","apt" : "아파트", "exclusive_use_area":"평수","year_of_completion" : "건축년도","floor":"층"}, inplace = True)
test_df.rename(columns = {'city' : '도시',"dong" : "동","apt" : "아파트", "exclusive_use_area":"평수","year_of_completion" : "건축년도","floor":"층"}, inplace = True)

<span style=color:red>
    <h2>
    필요한 훈련데이터만 추출(도시, 평수, 건축년도)
    </h2>
</span>

In [5]:
train_df_X = train_df.loc[:,["도시","평수","건축년도","층"]]
train_df_X["도시"] = train_df_X["도시"].apply(lambda x : 1 if x == "서울특별시" else 0)
test_df = test_df.loc[:,["도시","평수","건축년도","층"]]
test_df["도시"] = test_df["도시"].apply(lambda x : 1 if x == "서울특별시" else 0)

In [6]:
train_df_X.head(3)

,도시,평수,건축년도,층
0,1,84.82,2002,2
1,1,99.17,1973,6
2,1,84.74,2007,6


In [7]:
test_df.head(3)

,도시,평수,건축년도,층
0,1,35.55,2002,2
1,0,68.72,1977,2
2,0,72.54,1989,2


In [8]:
train_df_Y = train_df["transaction_real_price"] 
train_df_Y

0           37500
1           20000
2           38500
3          118000
4          120000
            ...  
1216548     22000
1216549     30750
1216550     29500
1216551     26700
1216552     28800
Name: transaction_real_price, Length: 1216553, dtype: int64

<span style=color:red>
    <h2>
    1. 선형회귀 모델 
    </h2>
</span>

In [59]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [77]:
lr.fit(train_df_X, train_df_Y)

LinearRegression()

In [78]:
result = lr.predict(test_df)

In [79]:
result[:10]

array([20089.21869464, 15314.3907522 , 15523.75371582, 68812.52103194,
       47860.09687879, -7920.64458472, 54200.51917027, 62164.95501676,
       65803.50116325, 33859.50057536])

In [80]:
result.min() 

-23374.932352510106

In [81]:
result.max()

163695.03001018552

<span style=color:red>
    <h2>
    2. 특성값이 적은 관계로, 특성을 늘려봄
    </h2>
</span>

In [82]:
from sklearn.preprocessing import PolynomialFeatures

In [83]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit(train_df_X)
train_poly = poly.transform(train_df_X)
test_df_poly = poly.transform(test_df)


In [84]:
lr.fit(train_poly,train_df_Y)

LinearRegression()

In [85]:
result = lr.predict(test_df_poly)
result[:10]

array([17763.08477688, 19953.23931885, 16467.76444721, 66426.57925797,
       57901.4422102 , 11319.33556557, 49698.5899334 , 65808.48897171,
       70230.58096027, 30867.37416649])

In [86]:
result.min() 

-28597.501110076904

In [88]:
result.max()

308596.3214826584

In [89]:
submission = pd.read_csv('submission.csv')
submission.head(3)

,transaction_id,transaction_real_price
0,1145756,0
1,1198704,0
2,1222384,0


In [90]:
for i in range(submission.shape[0]) :
    submission.loc[i,'transaction_real_price'] = result[i]

In [91]:
submission

,transaction_id,transaction_real_price
0,1145756,17763.084777
1,1198704,19953.239319
2,1222384,16467.764447
3,1179897,66426.579258
4,1223091,57901.442210
...,...,...
5458,1174640,44245.971659
5459,1175575,50366.897160
5460,1157024,77267.570939
5461,1136863,47077.593644


In [33]:
submission.to_csv('submission230221.csv', index=False)

<span style=color:red>
    <h2>
    특성을 더 늘려보자
    </h2>
</span>

In [126]:
day_care_center_df = pd.read_csv('day_care_center.csv')
day_care_center_df.head()

,city,gu,day_care_name,day_care_type,day_care_baby_num,teacher_num,nursing_room_num,playground_num,CCTV_num,is_commuting_vehicle,reference_date
0,서울특별시,서대문구,가람어린이집,국공립,70,11.0,5.0,1.0,7.0,N,2018-07-31
1,서울특별시,서대문구,가좌제일어린이집,국공립,88,22.0,5.0,2.0,10.0,N,2018-07-31
2,서울특별시,서대문구,경찰청어린이집,직장,90,23.0,6.0,1.0,32.0,N,2018-07-31
3,서울특별시,서대문구,고운햇살어린이집,가정,18,5.0,3.0,0.0,4.0,N,2018-07-31
4,서울특별시,서대문구,고은어린이집,국공립,93,20.0,7.0,2.0,10.0,N,2018-07-31


In [127]:
day_care_center_df = day_care_center_df[['city', 'gu', 'day_care_type', 'day_care_baby_num']]
day_care_center_df.head()

,city,gu,day_care_type,day_care_baby_num
0,서울특별시,서대문구,국공립,70
1,서울특별시,서대문구,국공립,88
2,서울특별시,서대문구,직장,90
3,서울특별시,서대문구,가정,18
4,서울특별시,서대문구,국공립,93


<span style=color:red>
    <h2>
    해야할 일
    </h2>
</span>

##### train_df 데이터에 구를 붙이기   
##### 어린이집 데이터를 더미 데이터로 만들기   
##### train_df 데이터구에 어린이집 붙이기

In [128]:
dummy_day_care_type = pd.get_dummies(day_care_center_df['day_care_type'], drop_first = False)
dummy_day_care_type

,가정,국공립,민간,법인·단체,사회복지법인,직장,협동
0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
7546,0,0,1,0,0,0,0
7547,1,0,0,0,0,0,0
7548,0,0,1,0,0,0,0
7549,0,0,1,0,0,0,0


In [129]:
dummy_day_care_type = dummy_day_care_type.add_prefix("어린이집유형_")
dummy_day_care_type

,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동
0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
7546,0,0,1,0,0,0,0
7547,1,0,0,0,0,0,0
7548,0,0,1,0,0,0,0
7549,0,0,1,0,0,0,0


In [130]:
day_care_center_df = pd.concat([day_care_center_df, dummy_day_care_type], axis = 1)
day_care_center_df

,city,gu,day_care_type,day_care_baby_num,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동
0,서울특별시,서대문구,국공립,70,0,1,0,0,0,0,0
1,서울특별시,서대문구,국공립,88,0,1,0,0,0,0,0
2,서울특별시,서대문구,직장,90,0,0,0,0,0,1,0
3,서울특별시,서대문구,가정,18,1,0,0,0,0,0,0
4,서울특별시,서대문구,국공립,93,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7546,부산광역시,금정구,민간,46,0,0,1,0,0,0,0
7547,부산광역시,금정구,가정,19,1,0,0,0,0,0,0
7548,부산광역시,금정구,민간,92,0,0,1,0,0,0,0
7549,부산광역시,금정구,민간,43,0,0,1,0,0,0,0


In [131]:
day_care_center_df.drop('day_care_type', axis = 1, inplace = True)
day_care_center_df

,city,gu,day_care_baby_num,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동
0,서울특별시,서대문구,70,0,1,0,0,0,0,0
1,서울특별시,서대문구,88,0,1,0,0,0,0,0
2,서울특별시,서대문구,90,0,0,0,0,0,1,0
3,서울특별시,서대문구,18,1,0,0,0,0,0,0
4,서울특별시,서대문구,93,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7546,부산광역시,금정구,46,0,0,1,0,0,0,0
7547,부산광역시,금정구,19,1,0,0,0,0,0,0
7548,부산광역시,금정구,92,0,0,1,0,0,0,0
7549,부산광역시,금정구,43,0,0,1,0,0,0,0


In [133]:
aggregated_day_care_center_df = day_care_center_df.groupby(['city', 'gu'], as_index = False)[day_care_center_df.columns[2:]].sum()
aggregated_day_care_center_df.head()

,city,gu,day_care_baby_num,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동
0,부산광역시,강서구,5312,71,6,35,0,5,5,1
1,부산광역시,금정구,5251,30,6,58,1,7,3,1
2,부산광역시,기장군,7689,98,13,56,0,5,3,3
3,부산광역시,남구,6676,65,12,50,2,6,6,2
4,부산광역시,동구,2214,8,9,11,2,2,7,0


<span style=color:red>
    <h2> 참조 데이터 불러오기</h2>
</span>    

In [9]:
addr_df = pd.read_excel("한국행정구역분류.xlsx",
                       sheet_name = "법정동코드 연계 자료분석용",
                       header = 1)

addr_df = addr_df.loc[addr_df['시도'].isin(['서울특별시', '부산광역시'])] # 서울특별시와 부산광역시 레코드만 가져옴
addr_df = addr_df[['시도', '시군구', '법정동']]
addr_df.head(5)



C:\Users\seo\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,시도,시군구,법정동
0,서울특별시,서울특별시,서울특별시
1,서울특별시,종로구,종로구
2,서울특별시,종로구,청운동
3,서울특별시,종로구,신교동
4,서울특별시,종로구,궁정동


In [10]:
addr_df["시도"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [12]:
train_df.head(5)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [13]:
train_df['dong']

0              신교동
1              필운동
2              필운동
3              내수동
4              내수동
            ...   
1216548    정관읍 용수리
1216549    정관읍 용수리
1216550    정관읍 용수리
1216551    정관읍 용수리
1216552    정관읍 용수리
Name: dong, Length: 1216553, dtype: object

In [15]:
addr_df_loc = addr_df['시도'] + '-' + addr_df['법정동']
train_df_loc = (train_df['city'] + '-' + train_df['dong']).unique()
addr_df_loc.head(5)

0    서울특별시-서울특별시
1      서울특별시-종로구
2      서울특별시-청운동
3      서울특별시-신교동
4      서울특별시-궁정동
dtype: object

In [18]:
train_df_loc[:5]

array(['서울특별시-신교동', '서울특별시-필운동', '서울특별시-내수동', '서울특별시-인의동', '서울특별시-이화동'],
      dtype=object)

In [19]:
import numpy as np
np.isin(train_df_loc, addr_df_loc)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [20]:
train_df_loc[~np.isin(train_df_loc, addr_df_loc)]

array(['부산광역시-기장읍 동부리', '부산광역시-기장읍 교리', '부산광역시-기장읍 서부리', '부산광역시-기장읍 대라리',
       '부산광역시-기장읍 청강리', '부산광역시-정관읍 달산리', '부산광역시-일광면 삼성리', '부산광역시-일광면 이천리',
       '부산광역시-기장읍 대변리', '부산광역시-정관읍 방곡리', '부산광역시-정관읍 용수리', '부산광역시-정관읍 매학리',
       '부산광역시-정관읍 모전리', '부산광역시-기장읍 내리', '부산광역시-철마면 고촌리', '부산광역시-장안읍 명례리',
       '부산광역시-장안읍 좌천리'], dtype=object)

In [21]:
addr_df.loc[addr_df['법정동'].str.contains('기장읍')]

,시도,시군구,법정동
1071,부산광역시,기장군,기장읍


In [22]:
# 결론: df의 dong에 리가 붙어있으면 제거해야 함
train_df['dong'] = train_df['dong'].str.split(' ', expand = True).iloc[:, 0]

# 재확인: 정상적으로 작동함을 확인
train_df_loc = (train_df['city'] + '-' + train_df['dong']).unique()
train_df_loc[~np.isin(train_df_loc, addr_df_loc)]

array([], dtype=object)

In [23]:
train_df = pd.merge(train_df, addr_df, left_on = ['city', 'dong'], right_on = ['시도', '법정동'])
train_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,시도,시군구,법정동
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,서울특별시,종로구,신교동
1,98,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,200803,11~20,2,39300,서울특별시,종로구,신교동
2,511,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200903,21~31,3,40200,서울특별시,종로구,신교동
3,840,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002,200909,1~10,1,42750,서울특별시,종로구,신교동
4,1201,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201011,21~30,3,51700,서울특별시,종로구,신교동
5,1302,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201101,1~10,4,53500,서울특별시,종로구,신교동
6,2657,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201312,21~31,3,50000,서울특별시,종로구,신교동
7,2754,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201402,11~20,3,50500,서울특별시,종로구,신교동
8,3533,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201503,1~10,3,48500,서울특별시,종로구,신교동
9,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,서울특별시,종로구,필운동


In [171]:
train_df.drop(['시도', '법정동'], axis = 1, inplace = True)
train_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,시군구
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,종로구
1,98,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,200803,11~20,2,39300,종로구
2,511,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200903,21~31,3,40200,종로구
3,840,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002,200909,1~10,1,42750,종로구
4,1201,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201011,21~30,3,51700,종로구
5,1302,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201101,1~10,4,53500,종로구
6,2657,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201312,21~31,3,50000,종로구
7,2754,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201402,11~20,3,50500,종로구
8,3533,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201503,1~10,3,48500,종로구
9,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,종로구


In [175]:
train_df['시군구코드'] = train_df['시군구'].astype("category").cat.codes

In [176]:
train_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,시군구,시군구코드
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500,종로구,35
1,98,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,200803,11~20,2,39300,종로구,35
2,511,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200903,21~31,3,40200,종로구,35
3,840,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002,200909,1~10,1,42750,종로구,35
4,1201,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201011,21~30,3,51700,종로구,35
5,1302,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201101,1~10,4,53500,종로구,35
6,2657,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201312,21~31,3,50000,종로구,35
7,2754,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201402,11~20,3,50500,종로구,35
8,3533,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201503,1~10,3,48500,종로구,35
9,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000,종로구,35


In [182]:
train_df = pd.merge(train_df, aggregated_day_care_center_df, left_on = ['city', '시군구'], right_on = ['city','gu'])
train_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,...,시군구코드,gu,day_care_baby_num,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,...,35,종로구,4902,4,33,9,3,2,27,0
1,98,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,200803,...,35,종로구,4902,4,33,9,3,2,27,0
2,511,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200903,...,35,종로구,4902,4,33,9,3,2,27,0
3,840,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002,200909,...,35,종로구,4902,4,33,9,3,2,27,0
4,1201,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201011,...,35,종로구,4902,4,33,9,3,2,27,0
5,1302,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201101,...,35,종로구,4902,4,33,9,3,2,27,0
6,2657,7623,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201312,...,35,종로구,4902,4,33,9,3,2,27,0
7,2754,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201402,...,35,종로구,4902,4,33,9,3,2,27,0
8,3533,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,201503,...,35,종로구,4902,4,33,9,3,2,27,0
9,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,...,35,종로구,4902,4,33,9,3,2,27,0


In [ ]:
# from tqdm import tqdm

# for i in tqdm(range(10000)):
#     print(i)

<span style=color:red>
    <h2>test전처리</h2>
</span>

In [186]:
test_df['dong'] = test_df['dong'].str.split(' ', expand = True).iloc[:, 0]

# 재확인: 정상적으로 작동함을 확인
test_df_loc = (test_df['city'] + '-' + test_df['dong']).unique()
test_df_loc[~np.isin(test_df_loc, addr_df_loc)]

array([], dtype=object)

In [187]:
test_df = pd.merge(test_df, addr_df, left_on = ['city', 'dong'], right_on = ['시도', '법정동'])
test_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,시도,시군구,법정동
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,서울특별시,양천구,목동
1,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,서울특별시,양천구,목동
2,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,서울특별시,양천구,목동
3,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,서울특별시,양천구,목동
4,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,서울특별시,양천구,목동
5,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,서울특별시,양천구,목동
6,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,서울특별시,양천구,목동
7,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,서울특별시,양천구,목동
8,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,서울특별시,양천구,목동
9,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,서울특별시,양천구,목동


In [188]:
test_df.drop(['시도', '법정동'], axis = 1, inplace = True)
test_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,시군구
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구
1,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구
2,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구
3,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구
4,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구
5,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구
6,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구
7,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구
8,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구
9,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구


<span style=color:red>
    <h2> 인덱싱
    from feature_engine.encoding import OneHotEncoder as OHE 아직 못함</h2>
</span>    

In [189]:
test_df['시군구코드'] = test_df['시군구'].astype("category").cat.codes
test_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,시군구,시군구코드
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구,29
1,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구,29
2,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구,29
3,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구,29
4,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,11~20,2,양천구,29
5,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구,29
6,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구,29
7,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구,29
8,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구,29
9,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,1~10,4,양천구,29


In [190]:
test_df = pd.merge(test_df, aggregated_day_care_center_df, left_on = ['city', '시군구'], right_on = ['city','gu'])
test_df.head(10)

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,...,시군구코드,gu,day_care_baby_num,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,...,29,양천구,12460,165,67,74,2,0,7,1
1,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,...,29,양천구,12460,165,67,74,2,0,7,1
2,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,...,29,양천구,12460,165,67,74,2,0,7,1
3,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,...,29,양천구,12460,165,67,74,2,0,7,1
4,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.55,2002,201711,...,29,양천구,12460,165,67,74,2,0,7,1
5,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,...,29,양천구,12460,165,67,74,2,0,7,1
6,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,...,29,양천구,12460,165,67,74,2,0,7,1
7,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,...,29,양천구,12460,165,67,74,2,0,7,1
8,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,...,29,양천구,12460,165,67,74,2,0,7,1
9,1146181,4419,서울특별시,목동,539-4,목동효성,목동 539-4 목동효성,84.80,2003,201712,...,29,양천구,12460,165,67,74,2,0,7,1


In [192]:
# train_df_X = train_df.loc[:,["도시","평수","건축년도","층"]]
train_df["city"] = train_df["city"].apply(lambda x : 1 if x == "서울특별시" else 0)
# test_df = test_df.loc[:,["도시","평수","건축년도","층"]]
test_df["city"] = test_df["city"].apply(lambda x : 1 if x == "서울특별시" else 0)

In [196]:
train_df['동코드'] = train_df['dong'].astype("category").cat.codes
test_df['동코드'] = train_df['dong'].astype("category").cat.codes

In [198]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,...,gu,day_care_baby_num,어린이집유형_가정,어린이집유형_국공립,어린이집유형_민간,어린이집유형_법인·단체,어린이집유형_사회복지법인,어린이집유형_직장,어린이집유형_협동,동코드
0,0,7622,1,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,...,종로구,4902,4,33,9,3,2,27,0,206
1,98,7623,1,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,200803,...,종로구,4902,4,33,9,3,2,27,0,206
2,511,7622,1,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200903,...,종로구,4902,4,33,9,3,2,27,0,206
3,840,7623,1,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002,200909,...,종로구,4902,4,33,9,3,2,27,0,206
4,1201,7623,1,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002,201011,...,종로구,4902,4,33,9,3,2,27,0,206


In [212]:
train_df_X = train_df.loc[:,["city","exclusive_use_area","year_of_completion","floor","어린이집유형_국공립","시군구코드"]]
test_df_X = test_df.loc[:,["city","exclusive_use_area","year_of_completion","floor","어린이집유형_국공립","시군구코드"]]

In [213]:
test_df_X.head()

,city,exclusive_use_area,year_of_completion,floor,어린이집유형_국공립,시군구코드
0,1,35.55,2002,2,67,29
1,1,35.55,2002,2,67,29
2,1,35.55,2002,2,67,29
3,1,35.55,2002,2,67,29
4,1,35.55,2002,2,67,29


In [214]:
train_df_Y = train_df["transaction_real_price"] 
train_df_Y

0          37500
1          39300
2          40200
3          42750
4          51700
           ...  
3626471    17100
3626472    18300
3626473    18600
3626474    17800
3626475    17800
Name: transaction_real_price, Length: 3626476, dtype: int64

In [215]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [216]:
lr.fit(train_df_X, train_df_Y)

LinearRegression()

In [217]:
result = lr.predict(test_df_X)
result[:10]

array([20694.90772668, 20694.90772668, 20694.90772668, 20694.90772668,
       20694.90772668, 50584.82297797, 50584.82297797, 50584.82297797,
       50584.82297797, 50584.82297797])

In [211]:
result.min()

-24187.525354275887

In [218]:
from tqdm import tqdm

submission = pd.read_csv('submission.csv')

for i in range(submission.shape[0]) :
    submission.loc[i,'transaction_real_price'] = result[i]



In [219]:
submission.to_csv('submission230221_ver.csv', index=False)

In [220]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit(train_df_X)
train_poly = poly.transform(train_df_X)
test_df_poly = poly.transform(test_df_X)

lr.fit(train_poly,train_df_Y)

result = lr.predict(test_df_poly)
result[:10]

array([14051.45674634, 14051.45674634, 14051.45674634, 14051.45674634,
       14051.45674634, 49007.6601088 , 49007.6601088 , 49007.6601088 ,
       49007.6601088 , 49007.6601088 ])

In [221]:
from tqdm import tqdm

submission = pd.read_csv('submission.csv')

for i in range(submission.shape[0]) :
    submission.loc[i,'transaction_real_price'] = result[i]
    
submission.to_csv('submission230221_ver.csv', index=False) 
print("완료")

완료


In [225]:
ls.score()

NameError: name 'ls' is not defined